In [ ]:
import sys
import regex as re
import numpy as np
import io
import matplotlib.pyplot as plt
import pandas as pd
import math
from numpy.linalg import norm
from scipy.spatial.distance import cosine
from sklearn.preprocessing import normalize
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Dropout, Lambda, TimeDistributed
from keras.layers import LSTM, Bidirectional, SimpleRNN, BatchNormalization, GRU
from keras.models import Model, load_model
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.activations import softmax
from copy import deepcopy
from google.colab import drive
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.ensemble import RandomForestClassifier
drive.mount('/content/drive', force_remount=True)


BASE_DIR = '/content/drive/My Drive/Colab Notebooks/thesis/'
INPUT_DIR = BASE_DIR + 'data/junyi_academy/raw/'
OUTPUT_DIR = BASE_DIR + 'data/junyi_academy/processed/sakt/'

TIME_STEPS = 300

!cp '/content/drive/My Drive/Colab Notebooks/thesis/lib/preprocessing.py' .
from preprocessing import process_sakt

Mounted at /content/drive


## Load the data

In [ ]:
# Remember that DataFrames are immutable
col_names = ['uuid', 'ucid', 'is_correct']
data = pd.read_csv(INPUT_DIR + "rawdata.csv", encoding = "cp850", usecols = col_names)

# drop rows that lack a skill_name
print(len(data))
data.columns

16217311


Index(['uuid', 'ucid', 'is_correct'], dtype='object')

## Create title to index dictionary

In [ ]:
categories = ['PADDING'] + list(set(data['ucid']))
number_of_exercises = len(categories)
index_to_category = dict(enumerate(categories))
category_to_index = { v:k for (k,v) in index_to_category.items() }

users = list(set(data['uuid']))
number_of_users = len(users)
index_to_user = dict(enumerate(users))
user_to_index = { v:k for (k,v) in index_to_user.items() }

print("Found %i categories" % len(categories))
print("Found %i users" % len(users))

Found 1327 categories
Found 72758 users


## Turn categories into indices

(This can take a while)

In [ ]:
# Columns are uuid (0), ucid (1) and is_correct (2)
total_rows = len(data)
for i in range(total_rows):
  data.iat[i, 0] = user_to_index[data.iat[i, 0]]
  data.iat[i, 1] = category_to_index[data.iat[i, 1]]
  if (i % 5000 == 0):
    sys.stdout.write('\r%i/%i (%.2f%%)' % (i, total_rows, i/total_rows*100))
    sys.stdout.flush()
sys.stdout.write('\r%i/%i (%.2f%%)' % (total_rows, total_rows, 100.))
sys.stdout.flush()

16217311/16217311 (100.00%)

In [ ]:
data = data.astype(int)

In [ ]:
data.to_csv(INPUT_DIR + "cleaned.csv")